!["Facultad de Ingeniería de la UBA"](logoFIUBA.jpg)


### Carrera de Maestría en Inteligencia Artificial (UBA) - _Cohorte 2_
## Análisis de Series Temporales I _(AST1)_
# **Trabajo Práctico 2:** _Linear Predictive Coding_

- #### Autor: Alejandro Lloveras _(a1716)_
- #### Docentes: 
    - Matias Vera
    - Camilo Argoty

### Ejercicio 1
Utilizando `load` (librosa), cargar el archivo de audio `estocastico.wav` y escucharlo utilizando `Audio` (IPython.display).

In [1]:
import librosa
import IPython.display

y, fs = librosa.load("estocastico.wav")
IPython.display.Audio(y, rate=fs)

### Ejercicio 2
Determinar aproximadamente los instantes de tiempo donde se pronuncian las vocales en el audio `estocastico.wav`. La función `recortar` de `recorte.py` puede ser útil.

In [2]:
from recorte import recortar
x,fs = librosa.load("estocastico.wav")
markers = recortar(x,fs)

FloatSlider(value=5000.0, description='Inicio:', max=39100.0, min=5000.0, step=100.0)

FloatSlider(value=6000.0, description='Fin:', max=39100.0, min=5000.0, step=100.0)

Button(description='Reproducir segmento', style=ButtonStyle())

Button(description='Agregar marcador', style=ButtonStyle())

Output()

Marcadores guardados (muestras):
1. **Vocal "E":** 13800 - 19700
2. **Vocal "O":** 19700 - 22500
3. **Vocal "A":** 22500 - 28000
4. **Vocal "I":** 28000 - 32000
5. **Vocal "O":** 32000 - 36000

---
## Linear Predictive Coding (LPC)

### Ejercicio 3
Utilizando `correlate` (numpy) implementar una función que calcule los coeficientes LPC de una señal (representativa de un solo sonido) para N > M.

In [ ]:
import numpy as np
def calc_lpc(y, m):
  """
  Calcula los M coeficientes LPC (h_k) para un segmento de señal y_segmento.
  Se utiliza el método de autocorrelación para resolver el sistema R * h = r.

  Args:
      y_segmento (np.ndarray): El segmento de señal (e.g., una vocal).
      M (int): El orden del modelo LPC (número de coeficientes a estimar).

  Returns:
      h (np.ndarray): Los M coeficientes LPC.
  """
  n = len(y)

  # Comprobación de condición N > M
  if n <= m:
      raise ValueError("La longitud de la señal (N) debe ser mayor que el orden LPC (M) para este método. CONDICIÓN: N > M")

  # Calculo de Autocorrelación Cruzada
  autocorr = np.correlate(y, y, mode='full') # longitud 2*n - 1
  autocorr = autocorr[n-1:]  # almacena lags positivos (k >= 0)
  estimador = autocorr / n  # estimador sesgado
  estimador = estimador[:m+1]  # nos quedamos con los primeros M+1 valores
  
  # Construcción de la matriz R y el vector r
  R = np.array([[estimador[abs(i - j)] for j in range(m)] for i in range(m)])  # matriz simétrica de autocorrelación
  r = estimador[1:]  # vector de autocorrelaciones para lags 1
  h = np.linalg.solve(R, r) # coeficientes LPC: h = R^(-1) * r.
  return h

In [ ]:
#TESTEO DE LA FUNCIÓN
max_len = 10 # Longitud máxima de la señal de prueba
signal = np.random.randint(-1e2, 1e2, size=np.random.randint(2, max_len)) # Señal aleatoria de prueba
print(f"Señal de prueba (N={len(signal)}):\n{signal}\n")
print("Estimadores LPC:\n",calc_lpc(signal, len(signal)-1)) # Debe devolver un array de N coeficientes

Señal de prueba (N=7):
[-43   0 -45  48 -46  20  23]

Estimadores LPC:
 [-0.45995146  0.25543833  0.15020372 -0.02051978 -0.19411427 -0.23886703]


### Ejercicio 4
Asumiendo conocidos los coeficientes LPC, hallar analíticamente la respuesta impulsiva de un sistema LTI cuya entrada sea $y[t]$ y su salida $x̂[t]$. Es decir, hallar $h_f[t]$ tal que $x̂[t] = (y * h_f)[t]$.

### Ejercicio 5
Utilizando `convolve` (numpy) implementar una función que estime la frecuencia de excitación a partir de la señal de un sonido y sus coeficientes LPC.

### Ejercicio 6
Asumiendo conocidos los coeficientes LPC, hallar una solución analítica para $H_f(e^{jω})$ (la respuesta en frecuencia del filtro del Ej. 4).

### Ejercicio 7
Asumiendo conocidos los coeficientes LPC, hallar una solución analítica para $H_i(e^{jω})$ (la respuesta en frecuencia del filtro definido en (5), donde $x̂[t]$ es la entrada y $ŷ[t]$ la salida).

### Ejercicio 8
Implementar una función que represente el sistema descripto en (5) (entrada $x̂[t]$ y salida $ŷ[t]$) a partir de su ecuación en diferencias.

**Nota:** No trate encontrar la respuesta impulsiva $h_i[t]$ definida en el Ej. 7 porque la misma posee duración infinita.

---
## Ventaneo _(Windowing)_

### Ejercicio 9
Utilizando la función desarrollada en el Ej. 3, calcular los coeficientes LPC del audio `estocastico.wav` para cada ventana de tiempo de la señal. Utilizar la ventana de `hamming` (numpy).

### Ejercicio 10
Utilizando `specgram` (matplotlib) graficar un espectrograma del audio `estocastico.wav`, configurando sus parámetros según lo discutido previamente.

### Ejercicio 11
Elegir los coeficientes LPC correspondientes a una ventana de tiempo de la letra /e/. Graficar $|H_f(e^{jω})|$ y $|H_i(e^{jω})|$.

### Ejercicio 12
Utilizando la función desarrollada en el Ej. 5, estimar la función de excitación $x̂[t]$ del audio `estocastico.wav` cuidando de evitar los transitorios del filtro. Esto puede hacerse conservando los transitorios finales de cada sección, y sumándolos al siguiente segmento.

### Ejercicio 13
Utilizando la función desarrollada en el Ej. 8, reconstruir la señal de habla $ŷ[t]$ y escucharla utilizando `Audio` (IPython.display).

---
## Ejercicios Optativos

### Ejercicio 14
Se desea reemplazar todas las vocales del audio con la letra /e/. Para ello, defina unos nuevos coeficientes LPC a partir de los originales. Reemplace todos los coeficientes correspondientes a vocales con los utilizados en el Ej. 11. Reconstruir la señal de habla (utilizando la señal $x̂[t]$ del Ej. 12) y escucharla utilizando `Audio` (IPython.display).

### Ejercicio 15
Se desea cambiar la frecuencia glótica del audio. Para ello se modificará $x̂[t]$ en los segmentos correspondientes a los fonemas sonoros (vocales en este caso). Cada segmento completo correspondiente a una vocal será duplicado (concatenando uno a continuación del otro) y quedándose con una de cada dos muestras (duplicando la frecuencia). Reconstruir la señal de habla (utilizando los coeficientes LPC del Ej. 9) y escucharla utilizando `Audio` (IPython.display).